In [ ]:
#change width of current notebook cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [1]:
import json;import datetime;from datetime import timedelta;import os, zipfile;
import pandas as pd;from scipy.optimize import curve_fit;from scipy.stats import chisquare
import numpy as np;import matplotlib.pyplot as plt
from LifeBuddyWebApp import create_app, db
from LifeBuddyWebApp.models import User, Post, Health_description, Health_measure
proxyApp=create_app()
ctx=proxyApp.app_context()
ctx.push()
from sqlalchemy import func

C:\tools\miniconda3\envs\lifebuddy\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [2]:
data_zip_med=r"C:\Users\captian2020\Downloads\polar-user-data-export_ce1e4395-ade2-4af6-9417-ee9a013faafb.zip"
polar_zip=zipfile.ZipFile(data_zip_med)

#read all files in zip to dictionary of file names and json (each value is a training_session)
polar_data_dict={}
for i in polar_zip.filelist:
    polar_data_dict[i.filename]=json.loads(polar_zip.read(i.filename))

In [ ]:
training_session=r"training-session-2021-03-21-5816145734-02a0a7b4-2b47-4ad5-9e41-94ff904ec5c0.json"
keys=polar_data_dict[training_session]['exercises'][0]['samples'].keys()
# for i,j in polar_data_dict[training_session]['exercises'][0]['samples'].items():
#     print(i, len(j))
polar_data_dict[training_session]['startTime']

In [ ]:
len(polar_data_dict)

Put polar data into dataframes

In [3]:
max_id=None
df_description=pd.DataFrame()
df_measure=pd.DataFrame()
for i,j in polar_data_dict.items():
    if 'training-session-' in i:
        file1 = open("training_session_file_tracker.txt","a")
        file1.write(i+"\n")
        file1.close()
        #get last id from database
        max_id=1 if max_id==None else max_id +1
        df1=pd.DataFrame()
        
        df1['var_activity']=[j['exercises'][0]['sport']]#if one row, first should be in brackets or soemthing liek that
        df1['var_type']='heart rate'#must come from input data polar h1 only does heart rate
        df1['var_periodicity']='seconds' #must come from input data polar h1 only does seconds
        df1['var_timezone_utc_delta_in_mins']=j['exercises'][0]['timezoneOffset']
        df1['time_stamp_utc']=datetime.datetime.utcnow()
        df1['user_id']=1#comes from app current_user.id
        df1['source_filename']=i
        df1['description_id']=max_id
        df1['datetime_of_activity']=datetime.datetime.strptime(j['startTime'],'%Y-%m-%dT%H:%M:%S.%f')

        var_datetime_utc_list=[]
        for data_item_series in j['exercises'][0]['samples']:
            
            
            if data_item_series!='recordedRoute' and len(var_datetime_utc_list)==0:
                var_datetime_utc_list=[datetime.datetime.strptime(
                    x['dateTime'],'%Y-%m-%dT%H:%M:%S.%f')+ timedelta(
                    minutes=j['exercises'][0]['timezoneOffset']) for x in j[
                    'exercises'][0]['samples'][data_item_series]]

                df2=pd.DataFrame(var_datetime_utc_list,columns=['var_datetime_utc'])
                df2['description_id']=max_id
                reading_count=len(var_datetime_utc_list)
            if data_item_series== 'heartRate':
                df2['heart_rate']=[x['value'] for x in j['exercises'][0]['samples'][data_item_series]]
            if data_item_series== 'speed':
                df2['speed']= [x['value'] for x in j['exercises'][0]['samples'][data_item_series]]
            if data_item_series== 'distance':
                df2['distance']= [x['value'] for x in j['exercises'][0]['samples'][data_item_series]]
            if data_item_series== 'recordedRoute':
                longitude_list= [x['longitude'] for x in j['exercises'][0]['samples'][data_item_series]]
                while reading_count>len(longitude_list):
                    longitude_list.append(longitude_list[-1])
                df2['longitude']=longitude_list
                latitude_list= [x['latitude'] for x in j['exercises'][0]['samples'][data_item_series]]
                while reading_count>len(latitude_list):
                    latitude_list.append(latitude_list[-1])

                df2['latitude']=latitude_list
                altitude_list= [x['altitude'] for x in j['exercises'][0]['samples'][data_item_series]]
                while reading_count>len(altitude_list):
                    altitude_list.append(altitude_list[-1])
                df2['altitude']=altitude_list
        df1['metric2_session_duration']=reading_count
        df_description=df_description.append(df1, ignore_index = True)
        df_measure=df_measure.append(df2, ignore_index = True)
df_description.set_index('description_id', inplace=True)

In [ ]:
df_description[df_description.datetime_of_activity==max(df_description.datetime_of_activity)]
# max(df_description.datetime_of_activity)

In [4]:
#The model set to max heartrate = 170
def michaelis_m_eq_fix170(time_var, shape_var):
    return (170 *time_var)/(shape_var + time_var)

In [10]:
#next part to loading data into datframes, but not exactly like in app
metric1_list=[]
a=df_measure['description_id'].unique()
for x in a:
    df_byId=df_measure[df_measure.description_id==x]
    x_obs=df_byId.var_datetime_utc.to_list()
    x_obs=[(i-x_obs[0]).total_seconds() for i in x_obs]
    y_obs=df_byId.heart_rate.to_list()
    popt_fix170 = curve_fit(michaelis_m_eq_fix170, x_obs, y_obs,bounds=(0,np.inf))[0][0]
    metric1_list.append(popt_fix170)

df_description['metric1_carido']=metric1_list

In [ ]:
#check to see if the app "add only new training sessions" is functioning correctly

In [5]:
####Check that same polar time stamps are not uploaded.
#get existing database health_measure.description id and var_datetime_utc ***TODO Make this filter on user*****
base_query_health_measure=db.session.query(Health_measure.description_id,Health_measure.var_datetime_utc)
health_measure_var_datetime=pd.read_sql(str(base_query_health_measure),db.session.bind) 
health_measure_var_datetime.rename(columns={'health_measure_description_id':'description_id',
                                           'health_measure_var_datetime_utc':'var_datetime_utc'}, inplace=True)

In [11]:
if len(health_measure_var_datetime)>0:
    ##convert both var_datetime_utc columns to string
    df_measure_2=df_measure
    df_measure_2.var_datetime_utc=df_measure_2.var_datetime_utc.astype(str)    

    #check that var_datetime_utc variable/column is same length as in df_measure
    if len(health_measure_var_datetime.var_datetime_utc[0])>len(df_measure_2.var_datetime_utc[0]):
        cut_length=(len(health_measure_var_datetime.var_datetime_utc[0])-len(df_measure_2.var_datetime_utc[0]))*-1
        health_measure_var_datetime.var_datetime_utc=health_measure_var_datetime.var_datetime_utc.str[:cut_length]
        print('health measures is longer')
    if len(df_measure_2.var_datetime_utc[0])>len(health_measure_var_datetime.var_datetime_utc[0]):
        cut_length=(len(df_measure_2.var_datetime_utc[0])-len(health_measure_var_datetime.var_datetime_utc[0]))*-1
        df_measure_2.var_datetime_utc=df_measure_2.var_datetime_utc.str[:cut_length]
        print('df measure 2 is longer')

    df_matching_times=pd.merge(df_measure_2, health_measure_var_datetime,on="var_datetime_utc")
    dup_descript_id_list=list(df_matching_times.description_id_x.unique())
    
    #overwrite upload dataframes with duplicate training sessions removed.
    df_measure=df_measure[~df_measure.description_id.isin(dup_descript_id_list)]
    df_description=df_description[~df_description.index.isin(dup_descript_id_list)]

health measures is longer


In [ ]:
df_description=pd.read_sql_table('health_description', db.engine)
df_description.head()

In [ ]:
df_measure=pd.read_sql_table('health_measure', db.engine)

In [ ]:
excelObj=pd.ExcelWriter("database_download2.xlsx")
wb1=excelObj.book
df_description.to_excel(excelObj,sheet_name='df_description', index=False)
df_measure.to_excel(excelObj,sheet_name='df_measure', index=False)
excelObj.close()

In [ ]:
#scratch work for loading data to dataframe dictinoaries

In [ ]:
max_id=None
test_dict={}
df_description=pd.DataFrame()
df_measure=pd.DataFrame()
for i,j in polar_data_dict.items():
    if 'training-session-' in i:
        #get last id from database
        max_id=1 if max_id==None else max_id +1
        print(max_id)
        temp_df_dict={}
        df1=pd.DataFrame()
        df1['description_id']=[max_id]

    df_description=df_description.append(df1, ignore_index = True)
df_description.set_index('description_id', inplace=True)

In [ ]:
df1=pd.DataFrame()
df1['description_id']=[max_id]

In [ ]:
df1=pd.DataFrame()
df1['var_activity']=[polar_data_dict[list(polar_data_dict.keys())[0]]['exercises'][0]['sport']]

In [ ]:
df1
# polar_data_dict[list(polar_data_dict.keys())[0]]['exercises'][0]['sport']

In [ ]:
df_byId=df_measure[df_measure.description_id==a[1]]
x_obs=df_byId.var_datetime_utc.to_list()
x_obs=[(i-var_datetime_utc_list[0]).total_seconds() for i in var_datetime_utc_list]
y_obs=df_byId.heart_rate.to_list()
popt_fix170 = curve_fit(michaelis_m_eq_fix170, x_obs, y_obs,bounds=(0,np.inf))[0][0]

In [ ]:
#put data into tables
df_description.to_sql('health_description',db.engine, if_exists='append',index=False)
df_measure.to_sql('health_measure',db.engine, if_exists='append',index=False)



calculate parameter for each session

In [ ]:
#get x and y series from each session
a=df_measure['description_id'].unique()
print(a)

In [ ]:
#append to database

In [ ]:
# base_query_health_measure=db.session.query(Health_description.id,Health_description.metric1_cardio)
db.session.query(Health_measure).all()

In [ ]:
df_measure.at[0,'var_datetime_utc']

In [ ]:
#delete all rows in a table
Health_description.query.delete()
Health_measure.query.delete()
db.session.commit()

In [ ]:
session=0

In [ ]:
# polar_data_dict['training-session-2021-03-25-5833693154-003b4bf2-e18f-46ad-87df-9254d8434062.json']['exercises'][0]['samples'].keys()
polar_data_dict[list(polar_data_dict.keys())[session]]['exercises'][0]['samples']['recordedRoute'][0]
# polar_data_dict.keys()

In [ ]:
polar_data_dict[list(polar_data_dict.keys())[session]]['exercises'][0]['samples']['heartRate'][0]

In [ ]:
polar_data_dict[list(polar_data_dict.keys())[session]]['exercises'][0]['samples']['speed'][-1]

In [ ]:
data_item_series=polar_data_dict['training-session-2021-03-25-5833693154-003b4bf2-e18f-46ad-87df-9254d8434062.json']['exercises'][0]['samples']['heartRate']
var_datetime_utc_list=[x['dateTime'] for x in data_item_series]

In [ ]:
#The model set to max heartrate = 170
def michaelis_m_eq_fix170(time_var, shape_var):
    return (170 *time_var)/(shape_var + time_var)

In [ ]:
def calc_rsq(obs_y, pred_y):
    residuals = obs_y - pred_y
    ss_res=np.sum(residuals**2)
    ss_tot=np.sum((obs_y-np.mean(obs_y))**2)
    r_squared=1-(ss_res/ss_tot)
    return r_squared

In [ ]:
polar_obs_var_dict={};polar_obs_var_dict_trim={}
polar_pred_var_dict={};polar_pred_var_dict_trim={}
polar_parameters_dict={};polar_parameters_dict_trim={}
polar_rsq_dict={};polar_rsq_dict_trim={}

for a,b in polar_df_dict2.items():

    #Get Observed x and y
    var_datetime_utc_list=polar_df_dict2[a].var_datetime_utc.to_list()
    obs_x=np.array([(i-var_datetime_utc_list[0]).total_seconds() for i in var_datetime_utc_list])
    obs_y=np.array(polar_df_dict2[a].var_value.to_list())
    obs_x_manuf=np.array(range(0,len(obs_y)))
    if len(obs_y)>140:
        obs_y=obs_y[:140]; obs_x=obs_x[:140]
    polar_obs_var_dict[a]=(obs_x, obs_y)
    
    #make trimmed observations based on minimum value of first 30 seconds of session
    min_value_30=polar_obs_var_dict[list(polar_obs_var_dict.keys())[0]][1][:30].min()
    min_position=np.where(polar_obs_var_dict[list(polar_obs_var_dict.keys())[0]][1][:30]==min_value_30)[0][-1]
    obs_y_trim=obs_y[min_position:]
    obs_x_trim=np.array(range(0,len(obs_y_trim)))
    if len(obs_y_trim)>140:
        obs_y_trim=obs_y_trim[:140]; obs_x_trim=obs_x_trim[:140]
    polar_obs_var_dict_trim[a]=(obs_x_trim, obs_y_trim)
    
    
    #calculate parameter for each model
    popt_fix170, pcov_fix170 = curve_fit(michaelis_m_eq_fix170, obs_x, obs_y,bounds=(0,np.inf))
    popt_fix170_trim, pcov_fix170_trim = curve_fit(michaelis_m_eq_fix170, obs_x_trim, obs_y_trim,bounds=(0,np.inf))
    polar_parameters_dict[a]=(popt_fix170[0])
    polar_parameters_dict_trim[a]=(popt_fix170_trim[0])

    
    #calculated predicted Y's for each model
    pred_y_fix170=[michaelis_m_eq_fix170(i, popt_fix170[0]) for i in obs_x]
    pred_y_fix170_trim=[michaelis_m_eq_fix170(i, popt_fix170_trim[0]) for i in obs_x_trim]
    polar_pred_var_dict[a]=( pred_y_fix170)
    polar_pred_var_dict_trim[a]=( pred_y_fix170_trim)
    
    #claculate the r square for each model
    rsq_fix170=calc_rsq(obs_y, pred_y_fix170)
    rsq_fix170_trim=calc_rsq(obs_y_trim, pred_y_fix170_trim)
    polar_rsq_dict[a]=(rsq_fix170)
    polar_rsq_dict_trim[a]=(rsq_fix170_trim)

In [ ]:
polar_df_dict1[list(polar_df_dict1.keys())[0]]

In [ ]:
#Dictionary of name, exercise, max obs_y, count
polar_data_dict_abbrev={}
for i,j in polar_df_dict1.items():
    max_value=polar_df_dict2[i].var_value.max()
    length_of_training=len(polar_df_dict2[i])
    polar_data_dict_abbrev[i]=(i,j.at[0,'var_activity'],max_value, length_of_training)

In [ ]:
df_description = pd.DataFrame.from_dict(polar_data_dict_abbrev, orient='index', columns=['name', 'exercise', 'maxObs_y', 'count'])
df_parameters = pd.DataFrame.from_dict(polar_parameters_dict, orient='index', columns=['popt_fix170'])
df_parameters_trim = pd.DataFrame.from_dict(polar_parameters_dict_trim, orient='index', columns=['popt_fix170_trim'])
df_rsq=pd.DataFrame.from_dict(polar_rsq_dict, orient='index', columns=['rsq_fix170'])
df_rsq_trim=pd.DataFrame.from_dict(polar_rsq_dict_trim, orient='index', columns=['rsq_fix170_trim'])

In [ ]:
#Make excel spreadsheet with polar_data_dict_abbrev | df_parameters |df_parameters_trim| df_rsq | df_rsq_trim
frames=[df_description, df_parameters, df_parameters_trim,df_rsq, df_rsq_trim]
result_primary=pd.concat(frames, axis=1)
result_primary.to_excel('big_table7_fix170_with_trim_max140.xlsx', index=False)

Check if sql table alraedy has time

In [ ]:
#get existing database health_measure.description id and var_datetime_utc
base_query_health_measure=db.session.query(Health_measure.description_id,Health_measure.var_datetime_utc)
health_measure_var_datetime=pd.read_sql(str(base_query_health_measure),db.session.bind) 
health_measure_var_datetime.rename(columns={'health_measure_description_id':'description_id',
                                           'health_measure_var_datetime_utc':'var_datetime_utc'}, inplace=True)

In [ ]:
##convert both var_datetime_utc columns to string
df_measure_2=df_measure
df_measure_2.var_datetime_utc=df_measure_2.var_datetime_utc.astype(str)

In [ ]:
#check that var_datetime_utc is same length as in df_measure
##put df_check_times first row into var and check length
if len(health_measure_var_datetime.var_datetime_utc[0])>len(df_measure_2.var_datetime_utc[0]):
    cut_length=(len(health_measure_var_datetime.var_datetime_utc[0])-len(df_measure_2.var_datetime_utc[0]))*-1
    health_measure_var_datetime.var_datetime_utc=health_measure_var_datetime.var_datetime_utc.str[:cut_length]
if len(df_measure_2.var_datetime_utc[0])>len(health_measure_var_datetime.var_datetime_utc[0]):
    cut_length=(len(df_measure_2.var_datetime_utc[0])-len(health_measure_var_datetime.var_datetime_utc[0]))*-1
    df_measure_2.var_datetime_utc=df_measure_2.var_datetime_utc.str[:cut_length]

In [ ]:
df_matching_times=pd.merge(df_measure_2, health_measure_var_datetime,on="var_datetime_utc")
df_matching_times.head()

In [ ]:
#if not same lenth: cut right most characters of df_check_times var_datatime_utc to match df_measures
#with new dataframes merge to a completely new dataframe
#get unique list of description_id's and remove from new data [df_measure and df_description]
dup_descript_id_list=list(df_matching_times.description_id_x.unique())

In [ ]:
df_measure[~df_measure.description_id.isin(dup_descript_id_list)]
df_description[~df_description.index.isin(dup_descript_id_list)]

In [ ]:
query2=db.session.query(Health_measure.description_id,Health_measure.var_datetime_utc)

In [ ]:
query2

In [ ]:
pd.read_sql(str(query2),db.session.bind) 

In [ ]:
query_str="""
SELECT health_measure.description_id AS health_measure_description_id, health_measure.var_datetime_utc AS health_measure_var_datetime_utc 
FROM health_measure 
WHERE health_measure.description_id = 1
"""

In [ ]:
df_check_times=pd.read_sql(query_str,db.session.bind)

In [ ]:
df_check_times.rename(columns={'health_measure_var_datetime_utc':'var_datetime_utc'}, inplace=True)

In [ ]:
df_check_times.var_datetime_utc.astype(str).head()
# df_check_times.var_datetime_utc[0]

In [ ]:
len(df_check_times.var_datetime_utc.str[:-3][0])

In [ ]:
df_check_times.var_datetime_utc=df_check_times.var_datetime_utc.str[:-3]
# str(df_measure.var_datetime_utc[0])

In [ ]:
df_matching_times=pd.merge(df_measure.var_datetime_utc.astype(str), df_check_times.var_datetime_utc.astype(str),on="var_datetime_utc")

In [ ]:
df_matching_times

In [ ]:
df_measure.head()

In [ ]:
df_check_times.head()

### scratch work start

In [ ]:
def plot_build(obs_x, obs_y, pred_y, rsq, param):
    plt.plot(obs_x, obs_y,':', label='Observed')
    plt.plot(obs_x,pred_y,'b-', label='predicted170')
    plt.legend(loc=4)
    plt.ylim([60, 185])
    n1='\n'
    plt.title(f"Shape Param:{param:3g}",x=0.4, y=0.9)

In [ ]:
session='training-session-2021-07-20-6287647724-d1b0e090-ef0a-4611-8adb-e85e5dc97ac2.json'
# obs_x, obs_y=polar_obs_var_dict[session]
obs_x, obs_y=polar_obs_var_dict_trim[session]
# param=polar_parameters_dict[session]
param_trim=polar_parameters_dict_trim[session]
pred_y_trim=polar_pred_var_dict_trim[session]
rsq_trim=polar_rsq_dict_trim[session]

plot_build(obs_x, obs_y, pred_y_trim, rsq_trim, param_trim)

In [ ]:
plot_build(obs_x, obs_y, obs_y2, rsq_trim, param_trim)

In [ ]:
plot_build(obs_x, pred_y_trim, pred_y_trim2, rsq_trim, param_trim)

In [ ]:
session='training-session-2021-07-25-6306180999-d111a1e8-6339-454f-bfa5-fc12773b3fb6.json'
# obs_x, obs_y=polar_obs_var_dict[session]
obs_x, obs_y2=polar_obs_var_dict_trim[session]
# param=polar_parameters_dict[session]
param_trim=polar_parameters_dict_trim[session]
pred_y_trim2=polar_pred_var_dict_trim[session]
rsq_trim=polar_rsq_dict_trim[session]

plot_build(obs_x, obs_y2, pred_y_trim2, rsq_trim, param_trim)

In [ ]:
session='training-session-2021-07-25-6306180999-d111a1e8-6339-454f-bfa5-fc12773b3fb6.json'
obs_x, obs_y=polar_obs_var_dict[session]
# obs_x, obs_y=polar_obs_var_dict_trim[session]
param=polar_parameters_dict[session]
param=polar_parameters_dict[session]
pred_y=polar_pred_var_dict[session]
rsq=polar_rsq_dict[session]

plot_build(obs_x, obs_y, pred_y, rsq, param)

### scratch work end

 # Start work for display/paper

### Approved plots

In [ ]:
#chart for self: observed and predicted
def plot_build_self(x, y_0, y_p, param_0):
    plt.plot(x, y_0,'k--', label='Observed')
    plt.plot(x,y_p,'k:', label='Predicted')
    plt.legend(loc=4)
    plt.ylim([60, 185])
    n1='\n'
    plt.title(f"Session_0 Param:{param_0:3g} (count:{len(x)})",x=0.05, y=0.9,fontdict={'horizontalalignment': 'left'})

In [ ]:
session='training-session-2021-07-25-6306180999-d111a1e8-6339-454f-bfa5-fc12773b3fb6.json'
x,y_0=polar_obs_var_dict[session]
y_p=polar_pred_var_dict[session]
param_0=polar_parameters_dict[session]
plot_build_self(x, y_0, y_p, param_0)

In [ ]:
#chart for green: observed
def plot_build_green_obs(x, obs_y_0, obs_y_h, param_0, param_h):
    plt.plot(x,obs_y_h,'g--', label='Healthier')
    plt.plot(x, obs_y_0,'k--', label='Less Health')
    plt.legend(loc=4)
    plt.ylim([60, 185])
    n1='\n'
    plt.title(f"Healthier Param:{param_h:3g} (count:{len(obs_x)}){n1}Less Health Param:{param_0:3g}",x=0.05, y=0.85,fontdict={'horizontalalignment': 'left'})

In [ ]:
session_h='training-session-2021-07-25-6306180999-d111a1e8-6339-454f-bfa5-fc12773b3fb6.json'
session_0='training-session-2021-06-25-6199038409-f9a41a7f-2200-4c89-8c33-fe964b45168c.json'

x,obs_y_0=polar_obs_var_dict[session_0]
x,obs_y_h=polar_obs_var_dict[session_h]
# y_p=polar_pred_var_dict[session]
param_0=polar_parameters_dict[session_0]
param_h=polar_parameters_dict[session_h]
plot_build_green_obs(x, obs_y_0, obs_y_h, param_0, param_h)

In [ ]:
session_h='training-session-2021-07-25-6306180999-d111a1e8-6339-454f-bfa5-fc12773b3fb6.json'
session_0='training-session-2021-06-25-6199038409-f9a41a7f-2200-4c89-8c33-fe964b45168c.json'

x,obs_y_0=polar_obs_var_dict[session_0]
x,obs_y_h=polar_obs_var_dict[session_h]

plt.plot(x,obs_y_h,'g--', label='25Jul2021')
plt.plot(x, obs_y_0,'k--', label='25Jun2021')
plt.legend(loc=4)
plt.ylim([60, 185])

In [ ]:
session_h='training-session-2021-07-25-6306180999-d111a1e8-6339-454f-bfa5-fc12773b3fb6.json'
session_0='training-session-2021-06-25-6199038409-f9a41a7f-2200-4c89-8c33-fe964b45168c.json'

# x,obs_y_0=polar_obs_var_dict[session_0]
# x,obs_y_h=polar_obs_var_dict[session_h]
y_h=polar_pred_var_dict[session]
y_0=polar_pred_var_dict[session_0]

param_0=polar_parameters_dict[session_0]
param_h=polar_parameters_dict[session_h]
n1='\n'
plt.plot(x,y_h,'g--', label='25Jul2021')
plt.plot(x, y_0,'k--', label='25Jun2021')
plt.legend(loc=4)
plt.ylim([60, 185])
plt.title(f"Healthier Param:{param_h:3g} (count:{len(obs_x)}){n1}Less Health Param:{param_0:3g}",x=0.05, y=0.85,fontdict={'horizontalalignment': 'left'})

In [ ]:
#chart for green: predicted
def plot_build_green_pred(x, y_0, y_h, param_0, param_h):
    plt.plot(x,y_h,'g:', label='Healthier')
    plt.plot(x, y_0,'k:', label='Less Health')
    plt.legend(loc=4)
    plt.ylim([60, 185])
#     plt.title(f"count:{len(obs_x)} /n param:{param_h:3g} /n param:{param_0:3g}",x=0.4, y=0.9)
    n1='\n'
    plt.title(f"Healthier Param:{param_h:3g} (count:{len(obs_x)}){n1}Less Health Param:{param_0:3g}",x=0.05, y=0.85,fontdict={'horizontalalignment': 'left'})

In [ ]:
session_h='training-session-2021-07-25-6306180999-d111a1e8-6339-454f-bfa5-fc12773b3fb6.json'
session_0='training-session-2021-06-25-6199038409-f9a41a7f-2200-4c89-8c33-fe964b45168c.json'

# x,obs_y_0=polar_obs_var_dict[session_0]
# x,obs_y_h=polar_obs_var_dict[session_h]
y_0=polar_pred_var_dict[session_0]
y_h=polar_pred_var_dict[session_h]
param_0=polar_parameters_dict[session_0]
param_h=polar_parameters_dict[session_h]
plot_build_green_pred(x, y_0, y_h, param_0, param_h)

In [ ]:
session_0='training-session-2021-06-25-6199038409-f9a41a7f-2200-4c89-8c33-fe964b45168c.json'
session_h='training-session-2021-07-31-6328565814-4dbab335-5784-4fc7-9bb3-ebe3d9d23e39.json'

x,obs_y_0=polar_obs_var_dict[session_0]
x,obs_y_h=polar_obs_var_dict[session_h]
# y_p=polar_pred_var_dict[session]
param_0=polar_parameters_dict[session_0]
param_h=polar_parameters_dict[session_h]
plot_build_green_obs(x, obs_y_0, obs_y_h, param_0, param_h)

In [ ]:
session_h='training-session-2021-07-31-6328565814-4dbab335-5784-4fc7-9bb3-ebe3d9d23e39.json'
session_0='training-session-2021-06-25-6199038409-f9a41a7f-2200-4c89-8c33-fe964b45168c.json'

# x,obs_y_0=polar_obs_var_dict[session_0]
# x,obs_y_h=polar_obs_var_dict[session_h]
y_0=polar_pred_var_dict[session_0]
y_h=polar_pred_var_dict[session_h]
param_0=polar_parameters_dict[session_0]
param_h=polar_parameters_dict[session_h]
plot_build_green_pred(x, y_0, y_h, param_0, param_h)

In [ ]:
session_h='training-session-2021-07-31-6328565814-4dbab335-5784-4fc7-9bb3-ebe3d9d23e39.json'
session_0='training-session-2021-07-25-6306180999-d111a1e8-6339-454f-bfa5-fc12773b3fb6.json'

# x,obs_y_0=polar_obs_var_dict[session_0]
# x,obs_y_h=polar_obs_var_dict[session_h]
y_0=polar_pred_var_dict[session_0]
y_h=polar_pred_var_dict[session_h]
param_0=polar_parameters_dict[session_0]
param_h=polar_parameters_dict[session_h]
plot_build_green_pred(x, y_0, y_h, param_0, param_h)

In [ ]:
session_h='training-session-2021-07-31-6328565814-4dbab335-5784-4fc7-9bb3-ebe3d9d23e39.json'
session_0='training-session-2021-05-16-6045582769-3f773490-3662-43aa-8969-c1e68091a47d.json'

x,obs_y_0=polar_obs_var_dict[session_0]
x,obs_y_h=polar_obs_var_dict[session_h]
# y_p=polar_pred_var_dict[session]
param_0=polar_parameters_dict[session_0]
param_h=polar_parameters_dict[session_h]
plot_build_green_obs(x, obs_y_0, obs_y_h, param_0, param_h)

In [ ]:
session_h='training-session-2021-07-31-6328565814-4dbab335-5784-4fc7-9bb3-ebe3d9d23e39.json'
session_0='training-session-2021-05-16-6045582769-3f773490-3662-43aa-8969-c1e68091a47d.json'
y_0=polar_pred_var_dict[session_0]
y_h=polar_pred_var_dict[session_h]
param_0=polar_parameters_dict[session_0]
param_h=polar_parameters_dict[session_h]
plot_build_green_pred(x, y_0, y_h, param_0, param_h)